In [10]:
import pandas as pd

# טוען את הקבצים
training_df = pd.read_csv("churn_training.csv")
holdout_df = pd.read_csv("churn_holdout.csv")

# המרת העמודה TotalCharges למספרים, שגיאות יהפכו ל-NaN
training_df['TotalCharges'] = pd.to_numeric(training_df['TotalCharges'], errors='coerce')
holdout_df['TotalCharges'] = pd.to_numeric(holdout_df['TotalCharges'], errors='coerce')

# מילוי ערכים חסרים בעמודות מספריות בעזרת החציוני
for col in ['MonthlyCharges', 'TotalCharges', 'tenure', 'SeniorCitizen']:
    training_df[col].fillna(training_df[col].median(), inplace=True)
    holdout_df[col].fillna(holdout_df[col].median(), inplace=True)

# מילוי ערכים חסרים בעמודות קטגוריאליות בעזרת הערך השכיח
categorical_cols = ['Partner', 'Dependents', 'InternetService', 'DeviceProtection',
                    'StreamingMovies', 'Contract', 'PaymentMethod']
for col in categorical_cols:
    training_df[col].fillna(training_df[col].mode()[0], inplace=True)

# בדיקה: האם נותרו ערכים חסרים?
print(training_df.isnull().sum())
print(holdout_df.isnull().sum())

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64
customerID             0
gender                 0
SeniorCitizen          0
Partner                0
Dependents             0
tenure                 0
PhoneService           0
MultipleLines          0
InternetService        0
OnlineSecurity         0
OnlineBackup           0
DeviceProtection       0
TechSupport            0
StreamingTV            0
StreamingMovies        0
Contract               0
PaperlessBilling       0
PaymentMethod          0
MonthlyCharges         0
TotalCharges           0
Churn               2045


In [12]:
from sklearn.model_selection import train_test_split

# משתנים מסבירים ותלויים
X = training_df.drop(columns=["customerID", "Churn"])
y = training_df["Churn"].map({"Yes": 1, "No": 0})  # המרה לערכים בינאריים

X_holdout = holdout_df.drop(columns=["customerID", "Churn"])

# פיצול ל־Train / Validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# משתנים קטגוריאליים וכמותיים
categorical_cols = X.select_dtypes(include="object").columns.tolist()
numeric_cols = X.select_dtypes(include=["float64", "int64"]).columns.tolist()

# Pipeline לעיבוד קטגוריאלי
categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Pipeline לעיבוד כמותי
numeric_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# מאחד את הכל
preprocessor = ColumnTransformer(transformers=[
    ("cat", categorical_pipeline, categorical_cols),
    ("num", numeric_pipeline, numeric_cols)
])

In [14]:
# מתאים את ה-preprocessing על סט האימון ומיישם גם על סט האימות
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)
X_holdout_processed = preprocessor.transform(X_holdout)

In [15]:
from sklearn.linear_model import LogisticRegression

# מאחד את כל סט האימון
X_full = training_df.drop(columns=["customerID", "Churn"])
y_full = training_df["Churn"].map({"Yes": 1, "No": 0})

# טרנספורמציה
X_full_processed = preprocessor.fit_transform(X_full)
X_holdout_processed = preprocessor.transform(holdout_df.drop(columns=["customerID", "Churn"]))

# אימון המודל הסופי
final_model = LogisticRegression(max_iter=1000)
final_model.fit(X_full_processed, y_full)

LogisticRegression(max_iter=1000)

In [16]:
# חיזוי על סט ההולד-אאוט
holdout_preds = final_model.predict(X_holdout_processed)

# יצירת DataFrame לפלט
submission = pd.DataFrame({
    "CustomerID": holdout_df["customerID"],
    "Prediction_Churn": holdout_preds
})

# שמירת הקובץ
submission.to_csv("submission.csv", index=False)
print("נשמר הקובץ: submission.csv")

נשמר הקובץ: submission.csv


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import pandas as pd

# יצירת מערך אימון מלא
X_full = training_df.drop(columns=["customerID", "Churn"])
y_full = training_df["Churn"].map({"Yes": 1, "No": 0})

# התאמת ה-preprocessor והמרת הנתונים
X_full_processed = preprocessor.fit_transform(X_full)
X_holdout_processed = preprocessor.transform(holdout_df.drop(columns=["customerID", "Churn"]))

# אימון המודל הסופי
final_model = LogisticRegression(max_iter=1000)
final_model.fit(X_full_processed, y_full)

# חיזוי על סט ה-holdout
holdout_preds = final_model.predict(X_holdout_processed)

# יצירת DataFrame עם התחזיות
submission = pd.DataFrame({
    "CustomerID": holdout_df["customerID"],
    "Prediction_Churn": holdout_preds
})

# שמירת קובץ התחזיות
submission.to_csv("submission.csv", index=False)

# חישוב F1 על סט האימות (validation)
val_preds = final_model.predict(X_val_processed)
val_f1 = f1_score(y_val, val_preds)

# הדפסת תוצאות
print(f"המודל חזה ש-{submission['Prediction_Churn'].sum()} מתוך {len(submission)} לקוחות ינטשו את החברה.")
print(f"F1 על סט האימות: {val_f1:.4f}")

המודל חזה ש-436 מתוך 2045 לקוחות ינטשו את החברה.
F1 על סט האימות: 0.5861
